**Important Libraries**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

**Reproducibility**

In [ ]:
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

**Paths**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR = "/content/drive/MyDrive/new work/Dataset/train"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
VAL_SPLIT = 0.20
NUM_CLASSES = None

Data generators
 - Train: augmentation + correct DenseNet preprocessing
 - Val: no augmentation + correct preprocessing

In [ ]:
train_datagen = ImageDataGenerator(
    validation_split=VAL_SPLIT,
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    validation_split=VAL_SPLIT,
    preprocessing_function=preprocess_input
)

train_it = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=SEED
)

val_it = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

class_names = list(train_it.class_indices.keys())
NUM_CLASSES = len(class_names)

Found 6079 images belonging to 4 classes.
Found 1518 images belonging to 4 classes.


In [ ]:
print("\nClass indices:", train_it.class_indices)
print("Classes:", class_names)


Class indices: {'Hyperpigmentation': 0, 'Nail fungus': 1, 'clubbing': 2, 'normal': 3}
Classes: ['Hyperpigmentation', 'Nail fungus', 'clubbing', 'normal']


In [ ]:
batchX, batchy = next(train_it)
print(f"\nBatch shape: {batchX.shape} | y shape: {batchy.shape}")
print(f"Batch min/max: {batchX.min():.3f}, {batchX.max():.3f}")


Batch shape: (32, 224, 224, 3) | y shape: (32, 4)
Batch min/max: -2.118, 2.640


Build model (transfer learning)
Stage 1: freeze backbone, train head
Stage 2: unfreeze top layers, fine-tune

In [ ]:
backbone = DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SIZE + (3,)
)

# Stage 1: freeze
backbone.trainable = False

x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.30)(x)
outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=backbone.input, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_4    │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d_4… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_5    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_5… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 18,329,668 (69.92 MB)

 Trainable params: 7,684 (30.02 KB)

 Non-trainable params: 18,321,984 (69.89 MB)

**Callbacks**

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.2, patience=2, min_lr=1e-7
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath="best_model.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

Train stage 1

In [31]:
EPOCHS_STAGE1 = 20
history1 = model.fit(
    train_it,
    validation_data=val_it,
    epochs=EPOCHS_STAGE1,
    callbacks=callbacks
)

Epoch 1/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 464s 2s/step - accuracy: 0.5680 - loss: 1.0324 - val_accuracy: 0.8702 - val_loss: 0.4088 - learning_rate: 0.0010
Epoch 2/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 114s 602ms/step - accuracy: 0.8540 - loss: 0.4311 - val_accuracy: 0.9097 - val_loss: 0.2822 - learning_rate: 0.0010
Epoch 3/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 116s 608ms/step - accuracy: 0.8919 - loss: 0.3180 - val_accuracy: 0.9321 - val_loss: 0.2211 - learning_rate: 0.0010
Epoch 4/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 112s 587ms/step - accuracy: 0.9101 - loss: 0.2702 - val_accuracy: 0.9440 - val_loss: 0.1832 - learning_rate: 0.0010
Epoch 5/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 115s 603ms/step - accuracy: 0.9117 - loss: 0.2639 - val_accuracy: 0.9565 - val_loss: 0.1577 - learning_rate: 0.0010
Epoch 6/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 119s 623ms/step - accuracy: 0.9173 - loss: 0.2354 - val_accuracy: 0.9565 - val_loss: 0.1527 - learning_rate: 0.0010
Epoch 7/20
190/190 ━━━━━━━━━━━━━━━━━━━━ 117s 615ms/step - accuracy:

KeyboardInterrupt: 

 Fine-tuning stage 2 (unfreeze top layers)

Unfreeze and keep earlier layers frozen for stability

In [ ]:
backbone.trainable = True

Freeze all layers except last N layers

In [ ]:
N_UNFROZEN = 60
for layer in backbone.layers[:-N_UNFROZEN]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

EPOCHS_STAGE2 = 10
history2 = model.fit(
    train_it,
    validation_data=val_it,
    epochs=EPOCHS_STAGE2,
    callbacks=callbacks
)

Plot training curves (combine histories)

In [ ]:

def combine_histories(h1, h2):
    out = {}
    for k in h1.history.keys():
        out[k] = h1.history[k] + h2.history.get(k, [])
    return out

hist = combine_histories(history1, history2)

plt.figure()
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])
plt.title("Model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "validation"], loc="lower right")
plt.show()

plt.figure()
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])
plt.title("Model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "validation"], loc="upper right")
plt.show()

Evaluation: predictions + confusion matrix + classification report

In [ ]:
# Reset generator
val_it.reset()

# Predictions
Y_pred = model.predict(val_it, verbose=1)
y_pred = np.argmax(Y_pred, axis=1)
y_true = val_it.classes

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

# High-quality figure
fig, ax = plt.subplots(figsize=(7, 6), dpi=300)

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=class_names
)

disp.plot(
    ax=ax,
    cmap="Blues",
    xticks_rotation=45,
    colorbar=True
)

ax.set_title("Confusion Matrix", fontsize=14)
ax.set_xlabel("Predicted Label", fontsize=12)
ax.set_ylabel("True Label", fontsize=12)

plt.tight_layout()
plt.show()

# Classification report (console only)
print("\nClassification Report:\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=class_names,
    digits=4
))

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

final metrics

In [ ]:
val_loss, val_acc = model.evaluate(val_it, verbose=0)
print(f"\nFinal Validation Accuracy: {val_acc:.4f}")
print(f"Final Validation Loss:     {val_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(hist["accuracy"]) + 1)

plt.figure(figsize=(9, 6), dpi=300)

plt.plot(epochs, hist["accuracy"], label="Training Accuracy", linewidth=2)
plt.plot(epochs, hist["val_accuracy"], label="Validation Accuracy", linewidth=2, linestyle="--")
plt.plot(epochs, hist["loss"], label="Training Loss", linewidth=2)
plt.plot(epochs, hist["val_loss"], label="Validation Loss", linewidth=2, linestyle="--")

plt.xlabel("Epoch")
plt.ylabel("Value")
plt.title("Training and Validation Metrics")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("training_validation_all_metrics.png", dpi=300, bbox_inches="tight")
plt.savefig("training_validation_all_metrics.pdf", bbox_inches="tight")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(hist["accuracy"]) + 1)

plt.figure(figsize=(8, 10), dpi=300)

# -------- Accuracy --------
plt.subplot(2, 1, 1)
plt.plot(epochs, hist["accuracy"], label="Training Accuracy", linewidth=2)
plt.plot(epochs, hist["val_accuracy"], label="Validation Accuracy", linewidth=2, linestyle="--")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.grid(True, alpha=0.3)

# -------- Loss --------
plt.subplot(2, 1, 2)
plt.plot(epochs, hist["loss"], label="Training Loss", linewidth=2)
plt.plot(epochs, hist["val_loss"], label="Validation Loss", linewidth=2, linestyle="--")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("training_validation_metrics.png", dpi=300, bbox_inches="tight")
plt.savefig("training_validation_metrics.pdf", bbox_inches="tight")
plt.show()
